<hr>
<h1>Predicting credit card approvals</h1>
<hr>
<p>Commercial banks receive a lot of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming. Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this notebook, I will build an automatic credit card approval predictor using machine learning techniques, just like the real banks do.</p>
<p><img src="assets/credit_card.jpg" alt="Credit card being held in hand"></p>
<p>I will use the <a href="http://archive.ics.uci.edu/ml/datasets/credit+approval">Credit Card Approval dataset</a> from the UCI Machine Learning Repository. The structure of this notebook is as follows:</p>
<ul>
    <li>First, I will start off by loading and viewing the dataset.</li>
    <li>I will see that the dataset has a mixture of both numerical and non-numerical features, that it contains values from different ranges, plus that it contains a number of missing entries.</li>
    <li>I will have to preprocess the dataset to ensure the machine learning model I choose can make good predictions.</li>
    <li>After the data is in good shape, I will do some exploratory data analysis.</li>
    <li>Finally, I will build a machine learning model that can predict if an individual's application for a credit card will be accepted.</li>
</ul>

<hr>
<h2>1. Setup</h2>
<hr>

In [17]:
# Import pandas
import pandas as pd

# Load dataset
cc_apps = pd.read_csv("assets/cc_approvals.data", header=None)

# Inspect data
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


<p>The output may appear a bit confusing at its first sight, but let's try to figure out the most important features of a credit card application. The features of this dataset have been anonymized to protect the privacy, but <a href="http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html">this blog</a> gives a pretty good overview of the probable features. The probable features in a typical credit card application are <code>Gender</code>, <code>Age</code>, <code>Debt</code>, <code>Married</code>, <code>BankCustomer</code>, <code>EducationLevel</code>, <code>Ethnicity</code>, <code>YearsEmployed</code>, <code>PriorDefault</code>, <code>Employed</code>, <code>CreditScore</code>, <code>DriversLicense</code>, <code>Citizen</code>, <code>ZipCode</code>, <code>Income</code>, and finally the <code>ApprovalStatus</code>. This gives me a pretty good starting point, and I can map these features with respect to the columns in the output. I can see from a first glance at the data, the dataset has a mixture of numerical and non-numerical features. This can be fixed with some preprocessing</p>

<hr>
<h2>2. Inspecting the applications</h2>
<hr>
<p>Before I do any preprocessing, let's learn about the dataset a bit more to see if there are other dataset issues that need to be fixed.</p>

In [18]:
# # Print summary statistics
# display(cc_apps.describe())

# # Print DataFrame information
# display(cc_apps.info())

# ”

Column: 0


b    468
a    210
?     12
Name: 0, dtype: int64


Column: 1


?        12
22.67     9
20.42     7
25.00     6
24.50     6
         ..
53.33     1
37.42     1
30.00     1
49.83     1
48.50     1
Name: 1, Length: 350, dtype: int64


Column: 2


...

00000    132
00120     35
00200     35
00160     34
00100     30
        ... 
00049      1
00076      1
00383      1
00030      1
00231      1
Name: 13, Length: 171, dtype: int64


Column: 14


0       295
1        29
1000     10
500      10
2         9
       ... 
2503      1
6590      1
1465      1
5552      1
283       1
Name: 14, Length: 240, dtype: int64


Column: 15


-    383
+    307
Name: 15, dtype: int64

<p>I have uncovered some issues that will affect the performance of the machine learning model(s) if they go unchanged:</p>
<ul>
    <li>The dataset contains both numeric and non-numeric data (specifically data that are of <code>float64</code>, <code>int64</code>, and <code>object</code> types). Specifically, the features 2, 7, 10 and 14 contain numeric values (of types <code>float64</code>, <code>float64</code>, <code>int64</code>, and <code>int64</code> respectively) and all the other features contain non-numeric values.</li>
    <li>The dataset also contains values from several ranges. Some features have a value range of 0 - 28, some have a range of 2 - 67, and some have a range of 1017 - 100000. Apart from these, I can get useful statistical information (like <code>mean</code>, <code>max</code>, and <code>min</code>) about the features that have numerical values.</li>
    <li>Finally, the dataset has missing values, which I will take care of in the next cell. The missing values in the dataset are labeled with '?', which can be seen if I run the code above.</li>
</ul>

<hr>
<h2>3. Handling the missing values (part 1)</h2>
<hr>
<p>Now, let's temporarily replace these missing value question marks with NaN.</p>

In [22]:
# Import numpy
import numpy as np

# Replace the '?'s with NaN
cc_apps = cc_apps.replace(to_replace="?", value=np.nan)

# # Inspect missing values again
# for col in cc_apps:
#     print(f"{'=' * 20}\nColumn: {col}\n{'=' * 20}")
#     display(cc_apps[col].value_counts())
#     print("")

<p>I replaced all the question marks with NaNs. This is going to help me in the next missing value treatment that I am going to perform.</p>
<p>An important question that could get raised here is <em>why am I giving so much importance to missing values</em>? Can't they be just ignored? Ignoring missing values can affect the performance of a machine learning model heavily. While ignoring the missing values our machine learning model may miss out on information about the dataset that may be useful for its training. Then, there are many models which cannot handle missing values implicitly such as LDA.</p>

<hr>
<h2>4. Handling the missing values (part 2)</h2>
<hr>
<p>To avoid the problem described in the previous cell, I an going to impute the missing values with a strategy called mean imputation.</p>

In [23]:
# Impute the missing values with mean imputation
cc_apps.fillna(cc_apps.mean(), inplace=True)

# Count the number of NaNs in the dataset to verify
cc_apps.isnull().sum().sum()

67

<p>I have successfully taken care of the missing values present in the numeric columns. There are still some missing values to be imputed for columns 0, 1, 3, 4, 5, 6 and 13. All of these columns contain non-numeric data and this why the mean imputation strategy would not work here. This needs a different treatment.</p>

<hr>
<h2>5. Handling the missing values (part iii)</h2>
<hr>
<p>I am going to impute these missing values with the most frequent values as present in the respective columns. This is <a href="https://www.datacamp.com/community/tutorials/categorical-data">good practice</a> when it comes to imputing missing values for categorical data in general.</p>

In [32]:
# Iterate over each column of cc_apps
for col in cc_apps.columns:
    # Check if the column is of object type
    if cc_apps[col].dtypes == "object":
        # Impute with the most frequent value
        cc_apps = cc_apps.fillna(cc_apps[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify
cc_apps.isnull().sum().sum()

0

<p>The missing values are now successfully handled.</p>
<p>There is still some minor but essential data preprocessing needed before I proceed towards building the machine learning model. I am going to divide these remaining preprocessing steps into three main tasks:</p>
<ol>
    <li>Convert the non-numeric data into numeric.</li>
    <li>Split the data into train and test sets.</li>
    <li>Scale the feature values to a uniform range.</li>
</ol>

<hr>
<h2>6. Preprocessing the data (part i)</h2>
<hr>
<p>First, I will be converting all the non-numeric values into numeric ones. I do this because not only it results in a faster computation but also many machine learning models (like XGBoost) (and especially the ones developed using scikit-learn) require the data to be in a strictly numeric format. I will do this by using a technique called <a href="http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html">label encoding</a>.</p>

In [36]:
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Instantiate LabelEncoder
le = LabelEncoder()

# Iterate over all the values of each column and extract their dtypes
for col in cc_apps.columns:
    # Compare if the dtype is object
    if cc_apps[col].dtype == "object":
    # Use LabelEncoder to do the numeric transformation
        x = le.fit(cc_apps.loc[:, col])
        cc_apps.loc[:, col] = x.transform(cc_apps.loc[:, col])
        
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,156,0.000,2,1,13,8,1.25,1,1,1,0,0,68,0,0
1,0,328,4.460,2,1,11,4,3.04,1,1,6,0,0,11,560,0
2,0,89,0.500,2,1,11,4,1.50,1,0,0,0,0,96,824,0
3,1,125,1.540,2,1,13,8,3.75,1,1,5,1,0,31,3,0
4,1,43,5.625,2,1,13,8,1.71,1,0,0,0,2,37,0,0


<p>Great, I have successfully converted all the non-numeric values to numeric ones.</p>

<hr>
<h2>7. Splitting the dataset into train and test sets</h2>
<p>Now, I will split the data into a train set and test set to prepare the data for two different phases of machine learning modeling: training and testing. Ideally, no information from the test data should be used to scale the training data or should be used to direct the training process of a machine learning model. Hence, I first split the data and then apply the scaling.</p>
<p>Also, features like <code>DriversLicense</code> and <code>ZipCode</code> are not as important as the other features in the dataset for predicting credit card approvals. I should drop them to design a machine learning model with the best set of features. In Data Science literature, this is often referred to as <em>feature selection</em>.</p>

In [45]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Drop the features 11 and 13 and convert the DataFrame to a NumPy array
cc_apps = cc_apps.drop([11, 13], axis=1)
cc_apps = cc_apps.values

# Segregate features and labels into separate variables
X, y = cc_apps[:, 0:13] , cc_apps[:, 13]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42
                                                   )

<p>The data is now split into two separate sets - train and test sets respectively. I am only left with one final preprocessing step of scaling before I can fit a machine learning model to the data.</p>

<hr>
<h2>8. Preprocessing the data (part ii)</h2>
<hr>
<p>Now, let's try to understand what these scaled values mean in the real world. Let's use <code>CreditScore</code> as an example. The credit score of a person is their creditworthiness based on their credit history. The higher this number, the more financially trustworthy a person is considered to be. So, a <code>CreditScore</code> of 1 is the highest since I am rescaling all the values to the range of 0-1.</p>

In [46]:
# Import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

<hr>
<h2>9. Fitting a logistic regression model to the train set</h2>
<hr>
<p>Essentially, predicting if a credit card application will be approved or not is a <a href="https://en.wikipedia.org/wiki/Statistical_classification">classification</a> task.<a href="http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.names">According to UCI</a>, the dataset contains more instances that correspond to "Denied" status than instances corresponding to "Approved" status. Specifically, out of 690 instances, there are 383 (55.5%) applications that got denied and 307 (44.5%) applications that got approved.</p>
<p>This gives me a benchmark. A good machine learning model should be able to accurately predict the status of the applications with respect to these statistics.</p>
<p>Which model should I pick? A question to ask is: <em>are the features that affect the credit card approval decision process correlated with each other?</em> Although I can measure correlation, that is outside the scope of this notebook, so I will rely on my intuition that they indeed are correlated for now. Because of this correlation, I will take advantage of the fact that generalized linear models perform well in these cases. Let's start the machine learning modeling with a Logistic Regression model (a generalized linear model).</p>

In [47]:
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

# Instantiate a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()

# Fit logreg to the train set
logreg.fit(rescaledX_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

<p>Great, but how well does this model perform?</p>

<hr>
<h2>10. Making predictions and evaluating performance</h2>
<hr>
<p>I will now evaluate the model on the test set with respect to <a href="https://developers.google.com/machine-learning/crash-course/classification/accuracy">classification accuracy</a>. But I will also take a look at the model's <a href="http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/">confusion matrix</a>. In the case of predicting credit card applications, it is equally important to see if our machine learning model is able to predict the approval status of the applications as denied that originally got denied. If our model is not performing well in this aspect, then it might end up approving the application that should have been denied. The confusion matrix helps us to view our model's performance from these aspects.</p>

In [49]:
# Import confusion_matrix
from sklearn.metrics import confusion_matrix

# Use logreg to predict instances from the test set and store it
y_pred = logreg.predict(rescaledX_test)

# Get the accuracy score of logreg model and print it
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test, y_test))

# Print the confusion matrix of the logreg model
display(confusion_matrix(y_test, y_pred))

Accuracy of logistic regression classifier:  0.8377192982456141


array([[92, 11],
       [26, 99]])

<p>The model is pretty good! It was able to yield an accuracy score of almost 84%.</p>
<p>For the confusion matrix, the first element of the of the first row of the confusion matrix denotes the true negatives meaning the number of negative instances (denied applications) predicted by the model correctly. And the last element of the second row of the confusion matrix denotes the true positives meaning the number of positive instances (approved applications) predicted by the model correctly.</p>

<hr>
<h2>11. Grid searching and making the model perform better</h2>
<hr>
<p>Let's see if there is a better model. I can perform a <a href="https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/">grid search</a> of the model parameters to improve the model's ability to predict credit card approvals.</p>
<p><a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">scikit-learn's implementation of logistic regression</a> consists of different hyperparameters but I will grid search over the following two:</p>
<ul>
    <li>tol</li>
    <li>max_iter</li>
</ul>

In [50]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Define the grid of values for tol and max_iter
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]

# Create a dictionary where tol and max_iter are keys and the lists of their values are corresponding values
param_grid = dict(tol=tol, max_iter=max_iter)

<p>I have defined the grid of hyperparameter values and converted them into a single dictionary format which <code>GridSearchCV()</code> expects as one of its parameters. Now, we will begin the grid search to see which values perform best.</p>

<hr>
<h2>12. Finding the best performing model</h2>
<hr>
<p>I will instantiate <code>GridSearchCV()</code> with the earlier <code>logreg</code> model with all the data we have. Instead of passing train and test sets separately, I will supply <code>X</code> (scaled version) and <code>y</code>. I will also instruct <code>GridSearchCV()</code> to perform a <a href="https://www.dataschool.io/machine-learning-with-scikit-learn/">cross-validation</a> of five folds.</p>
<p>I will end the notebook by storing the best-achieved score and the respective best parameters.</p>
<p>While building this credit card predictor, I tackled some of the most widely-known preprocessing steps such as <strong>scaling</strong>, <strong>label encoding</strong>, and <strong>missing value imputation</strong>. I finished with some <strong>machine learning</strong> to predict if a person's application for a credit card would get approved or not given some information about that person.</p>

In [51]:
# Instantiate GridSearchCV with the required parameters
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Use scaler to rescale X and assign it to rescaledX
rescaledX = scaler.fit_transform(X)

# Fit data to grid_model
grid_model_result = grid_model.fit(rescaledX, y)

# Summarize results
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print(f"Best: {best_score} using {best_params}")

Best: 0.8507246376811594 using {'max_iter': 100, 'tol': 0.01}
